### <font color='blue'>Import all packages</font> ###

In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

In [1]:

import tensorflow as tf
print(tf.__version__)

2.7.0


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import statistics
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization, Concatenate, Flatten
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-16 22:11:55.610743: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 22:11:57.946483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78949 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:4b:00.0, compute capability: 8.0
2022-09-16 22:11:57.957637: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/t

### <font color='blue'> Preprocessing and cleaning functions </font> ###

In [ ]:
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [3]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [4]:

def create_model():
    inps_first_512 = Input(shape = (max_len,), dtype='int64')
    inps_second_512 = Input(shape = (max_len,), dtype='int64')
    inps_third_512 = Input(shape = (max_len,), dtype='int64')
    inps_fourth_512 = Input(shape = (max_len,), dtype='int64')
    inps_fifth_512 = Input(shape = (max_len,), dtype='int64')
    inps_sixth_512 = Input(shape = (max_len,), dtype='int64')
    inps_seventh_512 = Input(shape = (max_len,), dtype='int64')

    masks_first_512= Input(shape = (max_len,), dtype='int64')
    masks_second_512= Input(shape = (max_len,), dtype='int64')
    masks_third_512= Input(shape = (max_len,), dtype='int64')
    masks_fourth_512= Input(shape = (max_len,), dtype='int64')
    masks_fifth_512= Input(shape = (max_len,), dtype='int64')
    masks_sixth_512= Input(shape = (max_len,), dtype='int64')
    masks_seventh_512= Input(shape = (max_len,), dtype='int64')

    dbert_layer_first_512 = dbert_model(inps_first_512, attention_mask=masks_first_512)[0][:,0,:]
    dbert_layer_second_512 = dbert_model(inps_second_512, attention_mask=masks_second_512)[0][:,0,:]
    dbert_layer_third_512 = dbert_model(inps_third_512, attention_mask=masks_third_512)[0][:,0,:]
    dbert_layer_fourth_512 = dbert_model(inps_fourth_512, attention_mask=masks_fourth_512)[0][:,0,:]
    dbert_layer_fifth_512 = dbert_model(inps_fifth_512, attention_mask=masks_fifth_512)[0][:,0,:]
    dbert_layer_sixth_512 = dbert_model(inps_sixth_512, attention_mask=masks_sixth_512)[0][:,0,:]
    dbert_layer_seventh_512 = dbert_model(inps_seventh_512, attention_mask=masks_seventh_512)[0][:,0,:]

    concat=Concatenate()([dbert_layer_first_512, dbert_layer_second_512,dbert_layer_third_512, dbert_layer_fourth_512,dbert_layer_fifth_512, dbert_layer_sixth_512,dbert_layer_seventh_512])
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.001))(concat)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps_first_512,masks_first_512,inps_second_512,masks_second_512,inps_third_512,masks_third_512,inps_fourth_512,masks_fourth_512,inps_fifth_512,masks_fifth_512,inps_sixth_512,masks_sixth_512,inps_seventh_512,masks_seventh_512], outputs=pred)
 
    
    print(model.summary())
    return model   

In [6]:
with strategy.scope():

    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts_complete=[]

    texts_0_512 = []  # list of text samples
    texts_448_960 = []  # list of text samples
    texts_896_1408 = []  # list of text samples
    texts_1344_1856 = []  # list of text samples
    texts_1792_2304 = []  # list of text samples
    texts_2240_2752 = []  # list of text samples
    texts_2688_3200 = []  # list of text samples

    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            #print(record)
            count=count+1
            #print(count)
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            first_512=new_new_sen[0:512]
            first_448_960=new_new_sen[448:960]
            first_512=' '.join(first_512)
            first_448_960=' '.join(first_448_960)

            first_896_1408=new_new_sen[896:1408]
            first_1344_1856=new_new_sen[1344:1856]
            first_896_1408=' '.join(first_896_1408)
            first_1344_1856=' '.join(first_1344_1856)

            first_1792_2304=new_new_sen[1792:2304]
            first_2240_2752=new_new_sen[2240:2752]
            first_1792_2304=' '.join(first_1792_2304)
            first_2240_2752=' '.join(first_2240_2752)

            first_2688_3200=new_new_sen[2688:3200]
            first_2688_3200=' '.join(first_2688_3200)

            texts_complete.append(new_sen)
            texts_0_512.append(first_512)
            texts_448_960.append(first_448_960)
            texts_896_1408.append(first_896_1408) 
            texts_1344_1856.append(first_1344_1856) 
            texts_1792_2304.append(first_1792_2304)  
            texts_2240_2752.append(first_2240_2752)
            texts_2688_3200.append(first_2688_3200) 

    len_list_complete = [len(ele.split()) for ele in texts_complete]
    len_list_0_512 = [len(ele.split()) for ele in texts_0_512]
    len_list_448_960 = [len(ele.split()) for ele in texts_448_960]
    len_list_896_1408 = [len(ele.split()) for ele in texts_896_1408]
    len_list_1344_1856 = [len(ele.split()) for ele in texts_1344_1856]
    len_list_1792_2304 = [len(ele.split()) for ele in texts_1792_2304]
    len_list_2240_2752 = [len(ele.split()) for ele in texts_2240_2752]
    len_list_2688_3200 = [len(ele.split()) for ele in texts_2688_3200]

    print(labels)
    print(len(labels))


    res_complete = 0 if len(len_list_complete) == 0 else (float(sum(len_list_complete)) / len(len_list_complete))
    print("Average Length of complete text %s" % res_complete) 
    print('Found %s texts in complete' % len(texts_complete))
    print("Median Length: %s " % statistics.median(len_list_complete))
    print("Max length of complete text %s" % max(len_list_complete))
    print("Min length of complete text %s" % min(len_list_complete))


    res_0_512 = 0 if len(len_list_0_512) == 0 else (float(sum(len_list_0_512)) / len(len_list_0_512))
    res_448_960 = 0 if len(len_list_448_960) == 0 else (float(sum(len_list_448_960)) / len(len_list_448_960))
    print("Average Length of First 512 %s" % res_0_512) 
    print("Average Length of Second 512 %s" % res_448_960) 
    print('Found %s texts in First 512.' % len(texts_0_512))
    print('Found %s texts in Second 512.' % len(texts_448_960))

    res_896_1408 = 0 if len(len_list_896_1408) == 0 else (float(sum(len_list_896_1408)) / len(len_list_896_1408))
    res_1344_1856 = 0 if len(len_list_1344_1856) == 0 else (float(sum(len_list_1344_1856)) / len(len_list_1344_1856))
    print("Average Length of Third 512 %s" % res_896_1408) 
    print("Average Length of Fourth 512 %s" % res_1344_1856) 
    print('Found %s texts in Third 512.' % len(texts_896_1408))
    print('Found %s texts in Fourth 512.' % len(texts_1344_1856))

    res_1792_2304 = 0 if len(len_list_1792_2304) == 0 else (float(sum(len_list_1792_2304)) / len(len_list_1792_2304))
    res_2240_2752 = 0 if len(len_list_2240_2752) == 0 else (float(sum(len_list_2240_2752)) / len(len_list_2240_2752))
    print("Average Length of Fifth 512 %s" % res_1792_2304) 
    print("Average Length of Sixth 512 %s" % res_2240_2752) 
    print('Found %s texts in Fifth 512.' % len(texts_1792_2304))
    print('Found %s texts in Sixth 512.' % len(texts_2240_2752))

    res_2688_3200 = 0 if len(len_list_2688_3200) == 0 else (float(sum(len_list_2688_3200)) / len(len_list_2688_3200))
    print("Average Length of Seventh 512 %s" % res_2688_3200)
    print('Found %s texts in Seventh 512.' % len(texts_2688_3200))


    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    print(label_list)
    label_list = label_list[0:-1]
    print(label_list)
    label_list = [int(i) for i in label_list]
    print('Found %s labels.' % len(set(label_list)))
    summarized_data = pd.DataFrame(list(zip(texts_0_512,texts_448_960,texts_896_1408,texts_1344_1856,texts_1792_2304,texts_2240_2752,texts_2688_3200)),
               columns =['First_512_text','Second_512_text','Third_512_text','Fourth_512_text','Fifth_512_text','Sixth_512_text', 'Seventh_512_text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    
    dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
    max_len=512
    first_512_sentences=summarized_data['First_512_text']
    second_512_sentences=summarized_data['Second_512_text']
    third_512_sentences=summarized_data['Third_512_text']
    fourth_512_sentences=summarized_data['Fourth_512_text']
    fifth_512_sentences=summarized_data['Fifth_512_text']
    sixth_512_sentences=summarized_data['Sixth_512_text']
    seventh_512_sentences=summarized_data['Seventh_512_text']
    labels=summarized_data['label']
    len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences),len(seventh_512_sentences)

    model_0=create_model()

    input_ids_first_512=[]
    attention_masks_first_512=[]
    for sent in summarized_data['First_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_first_512.append(dbert_inps['input_ids'])
        attention_masks_first_512.append(dbert_inps['attention_mask'])
    input_ids_first_512=np.asarray(input_ids_first_512)
    attention_masks_first_512=np.array(attention_masks_first_512)
    labels=np.array(labels)

    input_ids_second_512=[]
    attention_masks_second_512=[]
    for sent in summarized_data['Second_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_second_512.append(dbert_inps['input_ids'])
        attention_masks_second_512.append(dbert_inps['attention_mask'])
    input_ids_second_512=np.asarray(input_ids_second_512)
    attention_masks_second_512=np.array(attention_masks_second_512)

    input_ids_third_512=[]
    attention_masks_third_512=[]
    for sent in summarized_data['Third_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_third_512.append(dbert_inps['input_ids'])
        attention_masks_third_512.append(dbert_inps['attention_mask'])
    input_ids_third_512=np.asarray(input_ids_third_512)
    attention_masks_third_512=np.array(attention_masks_third_512)

    input_ids_fourth_512=[]
    attention_masks_fourth_512=[]
    for sent in summarized_data['Fourth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fourth_512.append(dbert_inps['input_ids'])
        attention_masks_fourth_512.append(dbert_inps['attention_mask'])
    input_ids_fourth_512=np.asarray(input_ids_fourth_512)
    attention_masks_fourth_512=np.array(attention_masks_fourth_512)

    input_ids_fifth_512=[]
    attention_masks_fifth_512=[]
    for sent in summarized_data['Fifth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fifth_512.append(dbert_inps['input_ids'])
        attention_masks_fifth_512.append(dbert_inps['attention_mask'])
    input_ids_fifth_512=np.asarray(input_ids_fifth_512)
    attention_masks_fifth_512=np.array(attention_masks_fifth_512)

    input_ids_sixth_512=[]
    attention_masks_sixth_512=[]
    for sent in summarized_data['Sixth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_sixth_512.append(dbert_inps['input_ids'])
        attention_masks_sixth_512.append(dbert_inps['attention_mask'])
    input_ids_sixth_512=np.asarray(input_ids_sixth_512)
    attention_masks_sixth_512=np.array(attention_masks_sixth_512)
    
    input_ids_seventh_512=[]
    attention_masks_seventh_512=[]
    for sent in summarized_data['Seventh_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_seventh_512.append(dbert_inps['input_ids'])
        attention_masks_seventh_512.append(dbert_inps['attention_mask'])
    input_ids_seventh_512=np.asarray(input_ids_seventh_512)
    attention_masks_seventh_512=np.array(attention_masks_seventh_512)

    train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512,train_inp_seventh_512,val_inp_seventh_512,train_mask_seventh_512,val_mask_seventh_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,input_ids_seventh_512,attention_masks_seventh_512,test_size=0.1,random_state=42)

    log_dir='dbert_model'

    model_save_path='Stride64/bert-stride64-512-0-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])


    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

2022-09-16 22:12:50.303205: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further tr

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 512)]        0           []                               
                                                                                              

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Fri Sep 16 22:19:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:4B:00.0 Off |                    0 |
| N/A   26C    P0    68W / 500W |  79806MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:E3:00.0 Off |                    0 |
| N/A   

In [8]:
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

In [43]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512,train_inp_seventh_512,train_mask_seventh_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512,val_inp_seventh_512,val_mask_seventh_512],val_label),callbacks=callbacks)

2022-09-16 18:30:07.282497: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_462013"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:56"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Y

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - ETA: 0s - loss: 2.4707 - accuracy: 0.6097

2022-09-16 18:40:36.117669: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_701679"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:85"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Y

948/948 [==============================] - 681s 456ms/step - loss: 2.4707 - accuracy: 0.6097 - val_loss: 2.0736 - val_accuracy: 0.6936
Epoch 2/5
948/948 [==============================] - 393s 414ms/step - loss: 1.9217 - accuracy: 0.7496 - val_loss: 1.8744 - val_accuracy: 0.7637
Epoch 3/5
948/948 [==============================] - 392s 414ms/step - loss: 1.6453 - accuracy: 0.8188 - val_loss: 1.8681 - val_accuracy: 0.7470
Epoch 4/5
948/948 [==============================] - 392s 414ms/step - loss: 1.4477 - accuracy: 0.8671 - val_loss: 1.8284 - val_accuracy: 0.7577
Epoch 5/5
948/948 [==============================] - 393s 414ms/step - loss: 1.2620 - accuracy: 0.9067 - val_loss: 1.8294 - val_accuracy: 0.7732


In [9]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('Stride64/bert-stride64-512-0-15labels.h5')

for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512),val_inp_seventh_512[i].reshape(1,512),val_mask_seventh_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_23 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
with strategy.scope():

    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts_complete=[]

    texts_0_512 = []  # list of text samples
    texts_448_960 = []  # list of text samples
    texts_896_1408 = []  # list of text samples
    texts_1344_1856 = []  # list of text samples
    texts_1792_2304 = []  # list of text samples
    texts_2240_2752 = []  # list of text samples
    texts_2688_3200 = []  # list of text samples

    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            #print(record)
            count=count+1
            #print(count)
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            first_512=new_new_sen[0:512]
            first_448_960=new_new_sen[448:960]
            first_512=' '.join(first_512)
            first_448_960=' '.join(first_448_960)

            first_896_1408=new_new_sen[896:1408]
            first_1344_1856=new_new_sen[1344:1856]
            first_896_1408=' '.join(first_896_1408)
            first_1344_1856=' '.join(first_1344_1856)

            first_1792_2304=new_new_sen[1792:2304]
            first_2240_2752=new_new_sen[2240:2752]
            first_1792_2304=' '.join(first_1792_2304)
            first_2240_2752=' '.join(first_2240_2752)

            first_2688_3200=new_new_sen[2688:3200]
            first_2688_3200=' '.join(first_2688_3200)

            texts_complete.append(new_sen)
            texts_0_512.append(first_512)
            texts_448_960.append(first_448_960)
            texts_896_1408.append(first_896_1408) 
            texts_1344_1856.append(first_1344_1856) 
            texts_1792_2304.append(first_1792_2304)  
            texts_2240_2752.append(first_2240_2752)
            texts_2688_3200.append(first_2688_3200) 

    len_list_complete = [len(ele.split()) for ele in texts_complete]
    len_list_0_512 = [len(ele.split()) for ele in texts_0_512]
    len_list_448_960 = [len(ele.split()) for ele in texts_448_960]
    len_list_896_1408 = [len(ele.split()) for ele in texts_896_1408]
    len_list_1344_1856 = [len(ele.split()) for ele in texts_1344_1856]
    len_list_1792_2304 = [len(ele.split()) for ele in texts_1792_2304]
    len_list_2240_2752 = [len(ele.split()) for ele in texts_2240_2752]
    len_list_2688_3200 = [len(ele.split()) for ele in texts_2688_3200]

    print(labels)
    print(len(labels))


    res_complete = 0 if len(len_list_complete) == 0 else (float(sum(len_list_complete)) / len(len_list_complete))
    print("Average Length of complete text %s" % res_complete) 
    print('Found %s texts in complete' % len(texts_complete))
    print("Median Length: %s " % statistics.median(len_list_complete))
    print("Max length of complete text %s" % max(len_list_complete))
    print("Min length of complete text %s" % min(len_list_complete))


    res_0_512 = 0 if len(len_list_0_512) == 0 else (float(sum(len_list_0_512)) / len(len_list_0_512))
    res_448_960 = 0 if len(len_list_448_960) == 0 else (float(sum(len_list_448_960)) / len(len_list_448_960))
    print("Average Length of First 512 %s" % res_0_512) 
    print("Average Length of Second 512 %s" % res_448_960) 
    print('Found %s texts in First 512.' % len(texts_0_512))
    print('Found %s texts in Second 512.' % len(texts_448_960))

    res_896_1408 = 0 if len(len_list_896_1408) == 0 else (float(sum(len_list_896_1408)) / len(len_list_896_1408))
    res_1344_1856 = 0 if len(len_list_1344_1856) == 0 else (float(sum(len_list_1344_1856)) / len(len_list_1344_1856))
    print("Average Length of Third 512 %s" % res_896_1408) 
    print("Average Length of Fourth 512 %s" % res_1344_1856) 
    print('Found %s texts in Third 512.' % len(texts_896_1408))
    print('Found %s texts in Fourth 512.' % len(texts_1344_1856))

    res_1792_2304 = 0 if len(len_list_1792_2304) == 0 else (float(sum(len_list_1792_2304)) / len(len_list_1792_2304))
    res_2240_2752 = 0 if len(len_list_2240_2752) == 0 else (float(sum(len_list_2240_2752)) / len(len_list_2240_2752))
    print("Average Length of Fifth 512 %s" % res_1792_2304) 
    print("Average Length of Sixth 512 %s" % res_2240_2752) 
    print('Found %s texts in Fifth 512.' % len(texts_1792_2304))
    print('Found %s texts in Sixth 512.' % len(texts_2240_2752))

    res_2688_3200 = 0 if len(len_list_2688_3200) == 0 else (float(sum(len_list_2688_3200)) / len(len_list_2688_3200))
    print("Average Length of Seventh 512 %s" % res_2688_3200)
    print('Found %s texts in Seventh 512.' % len(texts_2688_3200))


    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    print(label_list)
    label_list = label_list[0:-1]
    print(label_list)
    label_list = [int(i) for i in label_list]
    print('Found %s labels.' % len(set(label_list)))
    summarized_data = pd.DataFrame(list(zip(texts_0_512,texts_448_960,texts_896_1408,texts_1344_1856,texts_1792_2304,texts_2240_2752,texts_2688_3200)),
               columns =['First_512_text','Second_512_text','Third_512_text','Fourth_512_text','Fifth_512_text','Sixth_512_text', 'Seventh_512_text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    
    dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
    max_len=512
    first_512_sentences=summarized_data['First_512_text']
    second_512_sentences=summarized_data['Second_512_text']
    third_512_sentences=summarized_data['Third_512_text']
    fourth_512_sentences=summarized_data['Fourth_512_text']
    fifth_512_sentences=summarized_data['Fifth_512_text']
    sixth_512_sentences=summarized_data['Sixth_512_text']
    seventh_512_sentences=summarized_data['Seventh_512_text']
    labels=summarized_data['label']
    len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences),len(seventh_512_sentences)

    model_0=create_model()

    input_ids_first_512=[]
    attention_masks_first_512=[]
    for sent in summarized_data['First_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_first_512.append(dbert_inps['input_ids'])
        attention_masks_first_512.append(dbert_inps['attention_mask'])
    input_ids_first_512=np.asarray(input_ids_first_512)
    attention_masks_first_512=np.array(attention_masks_first_512)
    labels=np.array(labels)

    input_ids_second_512=[]
    attention_masks_second_512=[]
    for sent in summarized_data['Second_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_second_512.append(dbert_inps['input_ids'])
        attention_masks_second_512.append(dbert_inps['attention_mask'])
    input_ids_second_512=np.asarray(input_ids_second_512)
    attention_masks_second_512=np.array(attention_masks_second_512)

    input_ids_third_512=[]
    attention_masks_third_512=[]
    for sent in summarized_data['Third_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_third_512.append(dbert_inps['input_ids'])
        attention_masks_third_512.append(dbert_inps['attention_mask'])
    input_ids_third_512=np.asarray(input_ids_third_512)
    attention_masks_third_512=np.array(attention_masks_third_512)

    input_ids_fourth_512=[]
    attention_masks_fourth_512=[]
    for sent in summarized_data['Fourth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fourth_512.append(dbert_inps['input_ids'])
        attention_masks_fourth_512.append(dbert_inps['attention_mask'])
    input_ids_fourth_512=np.asarray(input_ids_fourth_512)
    attention_masks_fourth_512=np.array(attention_masks_fourth_512)

    input_ids_fifth_512=[]
    attention_masks_fifth_512=[]
    for sent in summarized_data['Fifth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fifth_512.append(dbert_inps['input_ids'])
        attention_masks_fifth_512.append(dbert_inps['attention_mask'])
    input_ids_fifth_512=np.asarray(input_ids_fifth_512)
    attention_masks_fifth_512=np.array(attention_masks_fifth_512)

    input_ids_sixth_512=[]
    attention_masks_sixth_512=[]
    for sent in summarized_data['Sixth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_sixth_512.append(dbert_inps['input_ids'])
        attention_masks_sixth_512.append(dbert_inps['attention_mask'])
    input_ids_sixth_512=np.asarray(input_ids_sixth_512)
    attention_masks_sixth_512=np.array(attention_masks_sixth_512)
    
    input_ids_seventh_512=[]
    attention_masks_seventh_512=[]
    for sent in summarized_data['Seventh_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_seventh_512.append(dbert_inps['input_ids'])
        attention_masks_seventh_512.append(dbert_inps['attention_mask'])
    input_ids_seventh_512=np.asarray(input_ids_seventh_512)
    attention_masks_seventh_512=np.array(attention_masks_seventh_512)

    train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512,train_inp_seventh_512,val_inp_seventh_512,train_mask_seventh_512,val_mask_seventh_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,input_ids_seventh_512,attention_masks_seventh_512,test_size=0.1,random_state=42)

    log_dir='dbert_model'

    model_save_path='Stride64/bert-stride64-512-1-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])


    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_36 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_30 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_37 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [51]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512,train_inp_seventh_512,train_mask_seventh_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512,val_inp_seventh_512,val_mask_seventh_512],val_label),callbacks=callbacks)

2022-09-16 19:22:43.326029: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1055297"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:11123"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this datase

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - ETA: 0s - loss: 2.4444 - accuracy: 0.6198

2022-09-16 19:33:11.945154: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1294963"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:11152"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this datase

948/948 [==============================] - 678s 452ms/step - loss: 2.4444 - accuracy: 0.6198 - val_loss: 2.0709 - val_accuracy: 0.6971
Epoch 2/5
948/948 [==============================] - 390s 411ms/step - loss: 1.9144 - accuracy: 0.7574 - val_loss: 1.8959 - val_accuracy: 0.7435
Epoch 3/5
948/948 [==============================] - 392s 413ms/step - loss: 1.6541 - accuracy: 0.8201 - val_loss: 1.8093 - val_accuracy: 0.7506
Epoch 4/5
948/948 [==============================] - 392s 414ms/step - loss: 1.4426 - accuracy: 0.8637 - val_loss: 1.8992 - val_accuracy: 0.7553
Epoch 5/5
948/948 [==============================] - 392s 414ms/step - loss: 1.2468 - accuracy: 0.9183 - val_loss: 1.7942 - val_accuracy: 0.7803


In [11]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('Stride64/bert-stride64-512-1-15labels.h5')

for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512),val_inp_seventh_512[i].reshape(1,512),val_mask_seventh_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_50 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_51 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
with strategy.scope():

    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts_complete=[]

    texts_0_512 = []  # list of text samples
    texts_448_960 = []  # list of text samples
    texts_896_1408 = []  # list of text samples
    texts_1344_1856 = []  # list of text samples
    texts_1792_2304 = []  # list of text samples
    texts_2240_2752 = []  # list of text samples
    texts_2688_3200 = []  # list of text samples

    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            #print(record)
            count=count+1
            #print(count)
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            
            first_512=new_new_sen[0:512]
            first_448_960=new_new_sen[448:960]
            first_512=' '.join(first_512)
            first_448_960=' '.join(first_448_960)

            first_896_1408=new_new_sen[896:1408]
            first_1344_1856=new_new_sen[1344:1856]
            first_896_1408=' '.join(first_896_1408)
            first_1344_1856=' '.join(first_1344_1856)

            first_1792_2304=new_new_sen[1792:2304]
            first_2240_2752=new_new_sen[2240:2752]
            first_1792_2304=' '.join(first_1792_2304)
            first_2240_2752=' '.join(first_2240_2752)

            first_2688_3200=new_new_sen[2688:3200]
            first_2688_3200=' '.join(first_2688_3200)

            texts_complete.append(new_sen)
            texts_0_512.append(first_512)
            texts_448_960.append(first_448_960)
            texts_896_1408.append(first_896_1408) 
            texts_1344_1856.append(first_1344_1856) 
            texts_1792_2304.append(first_1792_2304)  
            texts_2240_2752.append(first_2240_2752)
            texts_2688_3200.append(first_2688_3200) 

    len_list_complete = [len(ele.split()) for ele in texts_complete]
    len_list_0_512 = [len(ele.split()) for ele in texts_0_512]
    len_list_448_960 = [len(ele.split()) for ele in texts_448_960]
    len_list_896_1408 = [len(ele.split()) for ele in texts_896_1408]
    len_list_1344_1856 = [len(ele.split()) for ele in texts_1344_1856]
    len_list_1792_2304 = [len(ele.split()) for ele in texts_1792_2304]
    len_list_2240_2752 = [len(ele.split()) for ele in texts_2240_2752]
    len_list_2688_3200 = [len(ele.split()) for ele in texts_2688_3200]

    print(labels)
    print(len(labels))


    res_complete = 0 if len(len_list_complete) == 0 else (float(sum(len_list_complete)) / len(len_list_complete))
    print("Average Length of complete text %s" % res_complete) 
    print('Found %s texts in complete' % len(texts_complete))
    print("Median Length: %s " % statistics.median(len_list_complete))
    print("Max length of complete text %s" % max(len_list_complete))
    print("Min length of complete text %s" % min(len_list_complete))


    res_0_512 = 0 if len(len_list_0_512) == 0 else (float(sum(len_list_0_512)) / len(len_list_0_512))
    res_448_960 = 0 if len(len_list_448_960) == 0 else (float(sum(len_list_448_960)) / len(len_list_448_960))
    print("Average Length of First 512 %s" % res_0_512) 
    print("Average Length of Second 512 %s" % res_448_960) 
    print('Found %s texts in First 512.' % len(texts_0_512))
    print('Found %s texts in Second 512.' % len(texts_448_960))

    res_896_1408 = 0 if len(len_list_896_1408) == 0 else (float(sum(len_list_896_1408)) / len(len_list_896_1408))
    res_1344_1856 = 0 if len(len_list_1344_1856) == 0 else (float(sum(len_list_1344_1856)) / len(len_list_1344_1856))
    print("Average Length of Third 512 %s" % res_896_1408) 
    print("Average Length of Fourth 512 %s" % res_1344_1856) 
    print('Found %s texts in Third 512.' % len(texts_896_1408))
    print('Found %s texts in Fourth 512.' % len(texts_1344_1856))

    res_1792_2304 = 0 if len(len_list_1792_2304) == 0 else (float(sum(len_list_1792_2304)) / len(len_list_1792_2304))
    res_2240_2752 = 0 if len(len_list_2240_2752) == 0 else (float(sum(len_list_2240_2752)) / len(len_list_2240_2752))
    print("Average Length of Fifth 512 %s" % res_1792_2304) 
    print("Average Length of Sixth 512 %s" % res_2240_2752) 
    print('Found %s texts in Fifth 512.' % len(texts_1792_2304))
    print('Found %s texts in Sixth 512.' % len(texts_2240_2752))

    res_2688_3200 = 0 if len(len_list_2688_3200) == 0 else (float(sum(len_list_2688_3200)) / len(len_list_2688_3200))
    print("Average Length of Seventh 512 %s" % res_2688_3200)
    print('Found %s texts in Seventh 512.' % len(texts_2688_3200))


    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    print(label_list)
    label_list = label_list[0:-1]
    print(label_list)
    label_list = [int(i) for i in label_list]
    print('Found %s labels.' % len(set(label_list)))
    summarized_data = pd.DataFrame(list(zip(texts_0_512,texts_448_960,texts_896_1408,texts_1344_1856,texts_1792_2304,texts_2240_2752,texts_2688_3200)),
               columns =['First_512_text','Second_512_text','Third_512_text','Fourth_512_text','Fifth_512_text','Sixth_512_text', 'Seventh_512_text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    
    dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
    max_len=512
    first_512_sentences=summarized_data['First_512_text']
    second_512_sentences=summarized_data['Second_512_text']
    third_512_sentences=summarized_data['Third_512_text']
    fourth_512_sentences=summarized_data['Fourth_512_text']
    fifth_512_sentences=summarized_data['Fifth_512_text']
    sixth_512_sentences=summarized_data['Sixth_512_text']
    seventh_512_sentences=summarized_data['Seventh_512_text']
    labels=summarized_data['label']
    len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences),len(seventh_512_sentences)

    model_0=create_model()

    input_ids_first_512=[]
    attention_masks_first_512=[]
    for sent in summarized_data['First_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_first_512.append(dbert_inps['input_ids'])
        attention_masks_first_512.append(dbert_inps['attention_mask'])
    input_ids_first_512=np.asarray(input_ids_first_512)
    attention_masks_first_512=np.array(attention_masks_first_512)
    labels=np.array(labels)

    input_ids_second_512=[]
    attention_masks_second_512=[]
    for sent in summarized_data['Second_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_second_512.append(dbert_inps['input_ids'])
        attention_masks_second_512.append(dbert_inps['attention_mask'])
    input_ids_second_512=np.asarray(input_ids_second_512)
    attention_masks_second_512=np.array(attention_masks_second_512)

    input_ids_third_512=[]
    attention_masks_third_512=[]
    for sent in summarized_data['Third_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_third_512.append(dbert_inps['input_ids'])
        attention_masks_third_512.append(dbert_inps['attention_mask'])
    input_ids_third_512=np.asarray(input_ids_third_512)
    attention_masks_third_512=np.array(attention_masks_third_512)

    input_ids_fourth_512=[]
    attention_masks_fourth_512=[]
    for sent in summarized_data['Fourth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fourth_512.append(dbert_inps['input_ids'])
        attention_masks_fourth_512.append(dbert_inps['attention_mask'])
    input_ids_fourth_512=np.asarray(input_ids_fourth_512)
    attention_masks_fourth_512=np.array(attention_masks_fourth_512)

    input_ids_fifth_512=[]
    attention_masks_fifth_512=[]
    for sent in summarized_data['Fifth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fifth_512.append(dbert_inps['input_ids'])
        attention_masks_fifth_512.append(dbert_inps['attention_mask'])
    input_ids_fifth_512=np.asarray(input_ids_fifth_512)
    attention_masks_fifth_512=np.array(attention_masks_fifth_512)

    input_ids_sixth_512=[]
    attention_masks_sixth_512=[]
    for sent in summarized_data['Sixth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_sixth_512.append(dbert_inps['input_ids'])
        attention_masks_sixth_512.append(dbert_inps['attention_mask'])
    input_ids_sixth_512=np.asarray(input_ids_sixth_512)
    attention_masks_sixth_512=np.array(attention_masks_sixth_512)
    
    input_ids_seventh_512=[]
    attention_masks_seventh_512=[]
    for sent in summarized_data['Seventh_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_seventh_512.append(dbert_inps['input_ids'])
        attention_masks_seventh_512.append(dbert_inps['attention_mask'])
    input_ids_seventh_512=np.asarray(input_ids_seventh_512)
    attention_masks_seventh_512=np.array(attention_masks_seventh_512)

    train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512,train_inp_seventh_512,val_inp_seventh_512,train_mask_seventh_512,val_mask_seventh_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,input_ids_seventh_512,attention_masks_seventh_512,test_size=0.1,random_state=42)

    log_dir='dbert_model'

    model_save_path='Stride64/bert-stride64-512-2-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])


    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_57 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_64 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_58 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_65 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [54]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512,train_inp_seventh_512,train_mask_seventh_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512,val_inp_seventh_512,val_mask_seventh_512],val_label),callbacks=callbacks)

2022-09-16 20:10:21.255711: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1543311"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:22173"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this datase

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - ETA: 0s - loss: 2.4852 - accuracy: 0.5988

2022-09-16 20:20:54.449461: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1782977"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:22202"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this datase

948/948 [==============================] - 687s 458ms/step - loss: 2.4852 - accuracy: 0.5988 - val_loss: 2.1271 - val_accuracy: 0.7067
Epoch 2/5
948/948 [==============================] - 393s 414ms/step - loss: 1.9178 - accuracy: 0.7436 - val_loss: 1.8852 - val_accuracy: 0.7363
Epoch 3/5
948/948 [==============================] - 392s 414ms/step - loss: 1.6487 - accuracy: 0.8106 - val_loss: 1.8735 - val_accuracy: 0.7470
Epoch 4/5
948/948 [==============================] - 392s 414ms/step - loss: 1.4417 - accuracy: 0.8675 - val_loss: 1.8939 - val_accuracy: 0.7470
Epoch 5/5
948/948 [==============================] - 392s 414ms/step - loss: 1.2640 - accuracy: 0.9077 - val_loss: 1.8189 - val_accuracy: 0.7791


In [13]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('Stride64/bert-stride64-512-2-15labels.h5')

for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512),val_inp_seventh_512[i].reshape(1,512),val_mask_seventh_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

    

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_71 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_78 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_72 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_79 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts_complete=[]

    texts_0_512 = []  # list of text samples
    texts_448_960 = []  # list of text samples
    texts_896_1408 = []  # list of text samples
    texts_1344_1856 = []  # list of text samples
    texts_1792_2304 = []  # list of text samples
    texts_2240_2752 = []  # list of text samples
    texts_2688_3200 = []  # list of text samples

    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            #print(record)
            count=count+1
            #print(count)
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            first_512=new_new_sen[0:512]
            first_448_960=new_new_sen[448:960]
            first_512=' '.join(first_512)
            first_448_960=' '.join(first_448_960)

            first_896_1408=new_new_sen[896:1408]
            first_1344_1856=new_new_sen[1344:1856]
            first_896_1408=' '.join(first_896_1408)
            first_1344_1856=' '.join(first_1344_1856)

            first_1792_2304=new_new_sen[1792:2304]
            first_2240_2752=new_new_sen[2240:2752]
            first_1792_2304=' '.join(first_1792_2304)
            first_2240_2752=' '.join(first_2240_2752)

            first_2688_3200=new_new_sen[2688:3200]
            first_2688_3200=' '.join(first_2688_3200)

            texts_complete.append(new_sen)
            texts_0_512.append(first_512)
            texts_448_960.append(first_448_960)
            texts_896_1408.append(first_896_1408) 
            texts_1344_1856.append(first_1344_1856) 
            texts_1792_2304.append(first_1792_2304)  
            texts_2240_2752.append(first_2240_2752)
            texts_2688_3200.append(first_2688_3200) 

    len_list_complete = [len(ele.split()) for ele in texts_complete]
    len_list_0_512 = [len(ele.split()) for ele in texts_0_512]
    len_list_448_960 = [len(ele.split()) for ele in texts_448_960]
    len_list_896_1408 = [len(ele.split()) for ele in texts_896_1408]
    len_list_1344_1856 = [len(ele.split()) for ele in texts_1344_1856]
    len_list_1792_2304 = [len(ele.split()) for ele in texts_1792_2304]
    len_list_2240_2752 = [len(ele.split()) for ele in texts_2240_2752]
    len_list_2688_3200 = [len(ele.split()) for ele in texts_2688_3200]

    print(labels)
    print(len(labels))


    res_complete = 0 if len(len_list_complete) == 0 else (float(sum(len_list_complete)) / len(len_list_complete))
    print("Average Length of complete text %s" % res_complete) 
    print('Found %s texts in complete' % len(texts_complete))
    print("Median Length: %s " % statistics.median(len_list_complete))
    print("Max length of complete text %s" % max(len_list_complete))
    print("Min length of complete text %s" % min(len_list_complete))


    res_0_512 = 0 if len(len_list_0_512) == 0 else (float(sum(len_list_0_512)) / len(len_list_0_512))
    res_448_960 = 0 if len(len_list_448_960) == 0 else (float(sum(len_list_448_960)) / len(len_list_448_960))
    print("Average Length of First 512 %s" % res_0_512) 
    print("Average Length of Second 512 %s" % res_448_960) 
    print('Found %s texts in First 512.' % len(texts_0_512))
    print('Found %s texts in Second 512.' % len(texts_448_960))

    res_896_1408 = 0 if len(len_list_896_1408) == 0 else (float(sum(len_list_896_1408)) / len(len_list_896_1408))
    res_1344_1856 = 0 if len(len_list_1344_1856) == 0 else (float(sum(len_list_1344_1856)) / len(len_list_1344_1856))
    print("Average Length of Third 512 %s" % res_896_1408) 
    print("Average Length of Fourth 512 %s" % res_1344_1856) 
    print('Found %s texts in Third 512.' % len(texts_896_1408))
    print('Found %s texts in Fourth 512.' % len(texts_1344_1856))

    res_1792_2304 = 0 if len(len_list_1792_2304) == 0 else (float(sum(len_list_1792_2304)) / len(len_list_1792_2304))
    res_2240_2752 = 0 if len(len_list_2240_2752) == 0 else (float(sum(len_list_2240_2752)) / len(len_list_2240_2752))
    print("Average Length of Fifth 512 %s" % res_1792_2304) 
    print("Average Length of Sixth 512 %s" % res_2240_2752) 
    print('Found %s texts in Fifth 512.' % len(texts_1792_2304))
    print('Found %s texts in Sixth 512.' % len(texts_2240_2752))

    res_2688_3200 = 0 if len(len_list_2688_3200) == 0 else (float(sum(len_list_2688_3200)) / len(len_list_2688_3200))
    print("Average Length of Seventh 512 %s" % res_2688_3200)
    print('Found %s texts in Seventh 512.' % len(texts_2688_3200))


    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    print(label_list)
    label_list = label_list[0:-1]
    print(label_list)
    label_list = [int(i) for i in label_list]
    print('Found %s labels.' % len(set(label_list)))
    summarized_data = pd.DataFrame(list(zip(texts_0_512,texts_448_960,texts_896_1408,texts_1344_1856,texts_1792_2304,texts_2240_2752,texts_2688_3200)),
               columns =['First_512_text','Second_512_text','Third_512_text','Fourth_512_text','Fifth_512_text','Sixth_512_text', 'Seventh_512_text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    
    dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
    max_len=512
    first_512_sentences=summarized_data['First_512_text']
    second_512_sentences=summarized_data['Second_512_text']
    third_512_sentences=summarized_data['Third_512_text']
    fourth_512_sentences=summarized_data['Fourth_512_text']
    fifth_512_sentences=summarized_data['Fifth_512_text']
    sixth_512_sentences=summarized_data['Sixth_512_text']
    seventh_512_sentences=summarized_data['Seventh_512_text']
    labels=summarized_data['label']
    len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences),len(seventh_512_sentences)

    model_0=create_model()

    input_ids_first_512=[]
    attention_masks_first_512=[]
    for sent in summarized_data['First_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_first_512.append(dbert_inps['input_ids'])
        attention_masks_first_512.append(dbert_inps['attention_mask'])
    input_ids_first_512=np.asarray(input_ids_first_512)
    attention_masks_first_512=np.array(attention_masks_first_512)
    labels=np.array(labels)

    input_ids_second_512=[]
    attention_masks_second_512=[]
    for sent in summarized_data['Second_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_second_512.append(dbert_inps['input_ids'])
        attention_masks_second_512.append(dbert_inps['attention_mask'])
    input_ids_second_512=np.asarray(input_ids_second_512)
    attention_masks_second_512=np.array(attention_masks_second_512)

    input_ids_third_512=[]
    attention_masks_third_512=[]
    for sent in summarized_data['Third_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_third_512.append(dbert_inps['input_ids'])
        attention_masks_third_512.append(dbert_inps['attention_mask'])
    input_ids_third_512=np.asarray(input_ids_third_512)
    attention_masks_third_512=np.array(attention_masks_third_512)

    input_ids_fourth_512=[]
    attention_masks_fourth_512=[]
    for sent in summarized_data['Fourth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fourth_512.append(dbert_inps['input_ids'])
        attention_masks_fourth_512.append(dbert_inps['attention_mask'])
    input_ids_fourth_512=np.asarray(input_ids_fourth_512)
    attention_masks_fourth_512=np.array(attention_masks_fourth_512)

    input_ids_fifth_512=[]
    attention_masks_fifth_512=[]
    for sent in summarized_data['Fifth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fifth_512.append(dbert_inps['input_ids'])
        attention_masks_fifth_512.append(dbert_inps['attention_mask'])
    input_ids_fifth_512=np.asarray(input_ids_fifth_512)
    attention_masks_fifth_512=np.array(attention_masks_fifth_512)

    input_ids_sixth_512=[]
    attention_masks_sixth_512=[]
    for sent in summarized_data['Sixth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_sixth_512.append(dbert_inps['input_ids'])
        attention_masks_sixth_512.append(dbert_inps['attention_mask'])
    input_ids_sixth_512=np.asarray(input_ids_sixth_512)
    attention_masks_sixth_512=np.array(attention_masks_sixth_512)
    
    input_ids_seventh_512=[]
    attention_masks_seventh_512=[]
    for sent in summarized_data['Seventh_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_seventh_512.append(dbert_inps['input_ids'])
        attention_masks_seventh_512.append(dbert_inps['attention_mask'])
    input_ids_seventh_512=np.asarray(input_ids_seventh_512)
    attention_masks_seventh_512=np.array(attention_masks_seventh_512)

    train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512,train_inp_seventh_512,val_inp_seventh_512,train_mask_seventh_512,val_mask_seventh_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,input_ids_seventh_512,attention_masks_seventh_512,test_size=0.1,random_state=42)

    log_dir='dbert_model'

    model_save_path='Stride64/bert-stride64-512-3-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])


    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_85 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_92 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_86 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_93 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [15]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512,train_inp_seventh_512,train_mask_seventh_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512,val_inp_seventh_512,val_mask_seventh_512],val_label),callbacks=callbacks)

2022-09-16 22:49:04.463658: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_574113"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:32842"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


INFO:tensorflow:batch_all_reduce: 198 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:batch_all_reduce: 198 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then br

2022-09-16 22:59:37.342759: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_813783"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:32871"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset

948/948 [==============================] - 686s 456ms/step - loss: 2.4440 - accuracy: 0.6157 - val_loss: 2.1244 - val_accuracy: 0.6995
Epoch 2/5
948/948 [==============================] - 392s 413ms/step - loss: 1.9118 - accuracy: 0.7539 - val_loss: 1.8988 - val_accuracy: 0.7435
Epoch 3/5
948/948 [==============================] - 392s 413ms/step - loss: 1.6517 - accuracy: 0.8118 - val_loss: 1.7936 - val_accuracy: 0.7625
Epoch 4/5
948/948 [==============================] - 392s 413ms/step - loss: 1.4364 - accuracy: 0.8655 - val_loss: 1.8449 - val_accuracy: 0.7660
Epoch 5/5
948/948 [==============================] - 391s 413ms/step - loss: 1.2867 - accuracy: 0.9009 - val_loss: 1.9025 - val_accuracy: 0.7589


In [16]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('Stride64/bert-stride64-512-3-15labels.h5')

for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512),val_inp_seventh_512[i].reshape(1,512),val_mask_seventh_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 512)]        0           []                               
                                                                                              

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
with strategy.scope():

    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts_complete=[]

    texts_0_512 = []  # list of text samples
    texts_448_960 = []  # list of text samples
    texts_896_1408 = []  # list of text samples
    texts_1344_1856 = []  # list of text samples
    texts_1792_2304 = []  # list of text samples
    texts_2240_2752 = []  # list of text samples
    texts_2688_3200 = []  # list of text samples

    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            #print(record)
            count=count+1
            #print(count)
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            
            first_512=new_new_sen[0:512]
            first_448_960=new_new_sen[448:960]
            first_512=' '.join(first_512)
            first_448_960=' '.join(first_448_960)

            first_896_1408=new_new_sen[896:1408]
            first_1344_1856=new_new_sen[1344:1856]
            first_896_1408=' '.join(first_896_1408)
            first_1344_1856=' '.join(first_1344_1856)

            first_1792_2304=new_new_sen[1792:2304]
            first_2240_2752=new_new_sen[2240:2752]
            first_1792_2304=' '.join(first_1792_2304)
            first_2240_2752=' '.join(first_2240_2752)

            first_2688_3200=new_new_sen[2688:3200]
            first_2688_3200=' '.join(first_2688_3200)

            texts_complete.append(new_sen)
            texts_0_512.append(first_512)
            texts_448_960.append(first_448_960)
            texts_896_1408.append(first_896_1408) 
            texts_1344_1856.append(first_1344_1856) 
            texts_1792_2304.append(first_1792_2304)  
            texts_2240_2752.append(first_2240_2752)
            texts_2688_3200.append(first_2688_3200) 

    len_list_complete = [len(ele.split()) for ele in texts_complete]
    len_list_0_512 = [len(ele.split()) for ele in texts_0_512]
    len_list_448_960 = [len(ele.split()) for ele in texts_448_960]
    len_list_896_1408 = [len(ele.split()) for ele in texts_896_1408]
    len_list_1344_1856 = [len(ele.split()) for ele in texts_1344_1856]
    len_list_1792_2304 = [len(ele.split()) for ele in texts_1792_2304]
    len_list_2240_2752 = [len(ele.split()) for ele in texts_2240_2752]
    len_list_2688_3200 = [len(ele.split()) for ele in texts_2688_3200]

    print(labels)
    print(len(labels))


    res_complete = 0 if len(len_list_complete) == 0 else (float(sum(len_list_complete)) / len(len_list_complete))
    print("Average Length of complete text %s" % res_complete) 
    print('Found %s texts in complete' % len(texts_complete))
    print("Median Length: %s " % statistics.median(len_list_complete))
    print("Max length of complete text %s" % max(len_list_complete))
    print("Min length of complete text %s" % min(len_list_complete))


    res_0_512 = 0 if len(len_list_0_512) == 0 else (float(sum(len_list_0_512)) / len(len_list_0_512))
    res_448_960 = 0 if len(len_list_448_960) == 0 else (float(sum(len_list_448_960)) / len(len_list_448_960))
    print("Average Length of First 512 %s" % res_0_512) 
    print("Average Length of Second 512 %s" % res_448_960) 
    print('Found %s texts in First 512.' % len(texts_0_512))
    print('Found %s texts in Second 512.' % len(texts_448_960))

    res_896_1408 = 0 if len(len_list_896_1408) == 0 else (float(sum(len_list_896_1408)) / len(len_list_896_1408))
    res_1344_1856 = 0 if len(len_list_1344_1856) == 0 else (float(sum(len_list_1344_1856)) / len(len_list_1344_1856))
    print("Average Length of Third 512 %s" % res_896_1408) 
    print("Average Length of Fourth 512 %s" % res_1344_1856) 
    print('Found %s texts in Third 512.' % len(texts_896_1408))
    print('Found %s texts in Fourth 512.' % len(texts_1344_1856))

    res_1792_2304 = 0 if len(len_list_1792_2304) == 0 else (float(sum(len_list_1792_2304)) / len(len_list_1792_2304))
    res_2240_2752 = 0 if len(len_list_2240_2752) == 0 else (float(sum(len_list_2240_2752)) / len(len_list_2240_2752))
    print("Average Length of Fifth 512 %s" % res_1792_2304) 
    print("Average Length of Sixth 512 %s" % res_2240_2752) 
    print('Found %s texts in Fifth 512.' % len(texts_1792_2304))
    print('Found %s texts in Sixth 512.' % len(texts_2240_2752))

    res_2688_3200 = 0 if len(len_list_2688_3200) == 0 else (float(sum(len_list_2688_3200)) / len(len_list_2688_3200))
    print("Average Length of Seventh 512 %s" % res_2688_3200)
    print('Found %s texts in Seventh 512.' % len(texts_2688_3200))


    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    print(label_list)
    label_list = label_list[0:-1]
    print(label_list)
    label_list = [int(i) for i in label_list]
    print('Found %s labels.' % len(set(label_list)))
    summarized_data = pd.DataFrame(list(zip(texts_0_512,texts_448_960,texts_896_1408,texts_1344_1856,texts_1792_2304,texts_2240_2752,texts_2688_3200)),
               columns =['First_512_text','Second_512_text','Third_512_text','Fourth_512_text','Fifth_512_text','Sixth_512_text', 'Seventh_512_text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    
    dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
    max_len=512
    first_512_sentences=summarized_data['First_512_text']
    second_512_sentences=summarized_data['Second_512_text']
    third_512_sentences=summarized_data['Third_512_text']
    fourth_512_sentences=summarized_data['Fourth_512_text']
    fifth_512_sentences=summarized_data['Fifth_512_text']
    sixth_512_sentences=summarized_data['Sixth_512_text']
    seventh_512_sentences=summarized_data['Seventh_512_text']
    labels=summarized_data['label']
    len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences),len(seventh_512_sentences)

    model_0=create_model()

    input_ids_first_512=[]
    attention_masks_first_512=[]
    for sent in summarized_data['First_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_first_512.append(dbert_inps['input_ids'])
        attention_masks_first_512.append(dbert_inps['attention_mask'])
    input_ids_first_512=np.asarray(input_ids_first_512)
    attention_masks_first_512=np.array(attention_masks_first_512)
    labels=np.array(labels)

    input_ids_second_512=[]
    attention_masks_second_512=[]
    for sent in summarized_data['Second_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_second_512.append(dbert_inps['input_ids'])
        attention_masks_second_512.append(dbert_inps['attention_mask'])
    input_ids_second_512=np.asarray(input_ids_second_512)
    attention_masks_second_512=np.array(attention_masks_second_512)

    input_ids_third_512=[]
    attention_masks_third_512=[]
    for sent in summarized_data['Third_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_third_512.append(dbert_inps['input_ids'])
        attention_masks_third_512.append(dbert_inps['attention_mask'])
    input_ids_third_512=np.asarray(input_ids_third_512)
    attention_masks_third_512=np.array(attention_masks_third_512)

    input_ids_fourth_512=[]
    attention_masks_fourth_512=[]
    for sent in summarized_data['Fourth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fourth_512.append(dbert_inps['input_ids'])
        attention_masks_fourth_512.append(dbert_inps['attention_mask'])
    input_ids_fourth_512=np.asarray(input_ids_fourth_512)
    attention_masks_fourth_512=np.array(attention_masks_fourth_512)

    input_ids_fifth_512=[]
    attention_masks_fifth_512=[]
    for sent in summarized_data['Fifth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_fifth_512.append(dbert_inps['input_ids'])
        attention_masks_fifth_512.append(dbert_inps['attention_mask'])
    input_ids_fifth_512=np.asarray(input_ids_fifth_512)
    attention_masks_fifth_512=np.array(attention_masks_fifth_512)

    input_ids_sixth_512=[]
    attention_masks_sixth_512=[]
    for sent in summarized_data['Sixth_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_sixth_512.append(dbert_inps['input_ids'])
        attention_masks_sixth_512.append(dbert_inps['attention_mask'])
    input_ids_sixth_512=np.asarray(input_ids_sixth_512)
    attention_masks_sixth_512=np.array(attention_masks_sixth_512)
    
    input_ids_seventh_512=[]
    attention_masks_seventh_512=[]
    for sent in summarized_data['Seventh_512_text']:
        dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids_seventh_512.append(dbert_inps['input_ids'])
        attention_masks_seventh_512.append(dbert_inps['attention_mask'])
    input_ids_seventh_512=np.asarray(input_ids_seventh_512)
    attention_masks_seventh_512=np.array(attention_masks_seventh_512)

    train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512,train_inp_seventh_512,val_inp_seventh_512,train_mask_seventh_512,val_mask_seventh_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,input_ids_seventh_512,attention_masks_seventh_512,test_size=0.1,random_state=42)

    log_dir='dbert_model'

    model_save_path='Stride64/bert-stride64-512-4-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])


    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_23 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [18]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512,train_inp_seventh_512,train_mask_seventh_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512,val_inp_seventh_512,val_mask_seventh_512],val_label),callbacks=callbacks)

2022-09-16 23:36:23.394349: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1061522"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:43892"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this datase

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - ETA: 0s - loss: 2.4619 - accuracy: 0.6126

2022-09-16 23:46:51.983455: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1301188"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:43921"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this datase

948/948 [==============================] - 690s 466ms/step - loss: 2.4619 - accuracy: 0.6126 - val_loss: 2.0746 - val_accuracy: 0.6912
Epoch 2/5
948/948 [==============================] - 392s 414ms/step - loss: 1.9071 - accuracy: 0.7524 - val_loss: 1.8992 - val_accuracy: 0.7375
Epoch 3/5
948/948 [==============================] - 392s 414ms/step - loss: 1.6503 - accuracy: 0.8175 - val_loss: 1.8545 - val_accuracy: 0.7589
Epoch 4/5
948/948 [==============================] - 392s 414ms/step - loss: 1.4381 - accuracy: 0.8635 - val_loss: 1.8602 - val_accuracy: 0.7625
Epoch 5/5
948/948 [==============================] - 392s 414ms/step - loss: 1.2583 - accuracy: 0.9080 - val_loss: 1.7961 - val_accuracy: 0.7732


In [ ]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('Stride64/bert-stride64-512-4-15labels.h5')

for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512),val_inp_seventh_512[i].reshape(1,512),val_mask_seventh_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 512)]        0           []                               
                                                                                              

In [ ]:
print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))